# Test Python Loss Layer

In [1]:
import os
import scipy.io as io
import numpy as np
import sklearn
import sys
import matplotlib.pyplot as plt
%matplotlib inline 
import pylab

In [2]:
caffe_root = '/opt/caffe/'
import caffe

In [3]:
project_root = os.path.expanduser('/workspace/Weather/')
data_root = project_root

Net loading and setup

In [12]:
!cat /workspace/Weather/siamese/siamese_train_val.prototxt

name: "siamese_train_val"
layer {
  name: "data1"
  type: "Input"
  top: "data"
  input_param { shape: { dim: 1 dim: 3 dim: 227 dim: 227 } }
}
layer {
  name: "data2"
  type: "Input"
  top: "data_p"
  input_param { shape: { dim: 1 dim: 3 dim: 227 dim: 227 } }
}
layer {
  name: "ip1"
  type: "InnerProduct"
  bottom: "data"
  top: "ip1"
  param {
    name: "ip1_w"
    lr_mult: 1
  }
  param {
    name: "ip1_b"
    lr_mult: 2
  }
  inner_product_param {
    num_output: 2
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "ip1"
  top: "ip1"
}
layer {
  name: "ip2"
  type: "InnerProduct"
  bottom: "ip1"
  top: "ip2"
  param {
    name: "ip2_w"
    lr_mult: 1
  }
  param {
    name: "ip2_b"
    lr_mult: 2
  }
  inner_product_param {
    num_output: 1
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "ip1_p"
  type: "InnerProduct"


In [13]:
gpu_id = None
if gpu_id != None:
    caffe.set_device(0)  # if we have multiple GPUs, pick the first one
    caffe.set_mode_gpu()
else:
    caffe.set_mode_cpu()

In [14]:
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.get_solver('/workspace/Weather/siamese/solver.prototxt')

In [15]:
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

[('data', (1, 3, 227, 227)),
 ('data_p', (1, 3, 227, 227)),
 ('ip1', (1, 2)),
 ('ip2', (1, 1)),
 ('ip1_p', (1, 2)),
 ('ip2_p', (1, 1)),
 ('loss', (1,))]

In [16]:
solver.net.blobs['data'].data[...] = np.random.rand(1, 1, 227, 227)
solver.net.blobs['data_p'].data[...] = np.random.rand(1, 1, 227, 227)

In [17]:
solver.net.forward()  # train net

(1, 1)


{'loss': array([ 0.52990818], dtype=float32)}

In [18]:
print solver.net.blobs['ip2'].data
print solver.net.blobs['ip2_p'].data

[[ 0.04412685]]
[[ 0.16390245]]


In [19]:
solver.net.backward()   # train net

2
bottom  0  is  (1, 1)
bottom  1  is  (1, 1)


{'data': array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]]], dtype=float32),
 'data_p': array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0., 

In [23]:
print solver.net.blobs['ip2'].diff
print solver.net.blobs['ip2_p'].diff

[[-0.2491055]]
[[ 0.2491055]]
